<a href="https://colab.research.google.com/github/Sowmya-2348562/LLM/blob/main/Sowmya_562_Lab4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.1/316.1 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.4 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.6.1
    Uninstalling fsspec-2024.6.1:
     

In [ ]:
from datasets import load_dataset
dataset=load_dataset('imdb')

In [ ]:
import random
postive = [x for x in dataset['train'] if x['label'] == 1][:2500]
negative = [x for x in dataset['train'] if x['label'] == 0][:2500]
train_dataset = postive + negative
random.shuffle(train_dataset)
positive_test = [x for x in dataset['test'] if x['label'] == 1][:2500]
negative_test = [x for x in dataset['test'] if x['label'] == 0][:2500]
test_dataset = positive_test + negative_test
random.shuffle(test_dataset)

Loading Tokenizer and BERT model

In [ ]:
from transformers import BertTokenizer, TFBertForSequenceClassification
tokenizer=BertTokenizer.from_pretrained('bert-base-uncased')
model=TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

Defining Train and Test data

In [ ]:

train_texts = [x['text'] for x in train_dataset]
train_labels = [x['label'] for x in train_dataset]
test_texts = [x['text'] for x in test_dataset]
test_labels = [x['label'] for x in test_dataset]


Tokenizing train and test test

In [ ]:

import tensorflow as tf
train_encodings=tokenizer(train_texts, truncation=True, padding=True, max_length=128)
test_encodings=tokenizer(test_texts, truncation=True, padding=True, max_length=128)


Mapping train and test to labels

In [ ]:

train_dataset=tf.data.Dataset.from_tensor_slices((dict(train_encodings), train_labels))
test_dataset=tf.data.Dataset.from_tensor_slices((dict(test_encodings), test_labels))
train_dataset=train_dataset.shuffle(1000).batch(8).prefetch(tf.data.AUTOTUNE)
test_dataset=test_dataset.batch(8).prefetch(tf.data.AUTOTUNE)

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])


In [ ]:
model.fit(train_dataset, epochs=3, validation_data=test_dataset)

Epoch 1/3
625/625 [==============================] - 249s 326ms/step - loss: 0.3836 - accuracy: 0.8242 - val_loss: 0.3157 - val_accuracy: 0.8652
Epoch 2/3
625/625 [==============================] - 202s 323ms/step - loss: 0.1916 - accuracy: 0.9276 - val_loss: 0.3710 - val_accuracy: 0.8698
Epoch 3/3
625/625 [==============================] - 202s 323ms/step - loss: 0.0961 - accuracy: 0.9688 - val_loss: 0.4720 - val_accuracy: 0.8584


In [ ]:

loss, accuracy = model.evaluate(test_dataset)
print(f"Test Accuracy: {accuracy:.2f}")

625/625 [==============================] - 52s 83ms/step - loss: 0.4720 - accuracy: 0.8584
Test Accuracy: 0.86


Sentiment Prediction function

In [ ]:
def predict_sentiment(text):
    inputs = tokenizer(text, return_tensors="tf", truncation=True, padding=True, max_length=128)
    outputs = model(inputs)
    logits = outputs.logits
    predictions = tf.nn.softmax(logits, axis=-1)
    sentiment = tf.argmax(predictions, axis=1).numpy()[0]
    return sentiment

In [ ]:

text = input("Enter text for sentiment analysis: ")
sentiment = predict_sentiment(text)
sentiment_label = "Positive" if sentiment == 1 else "Negative"
print(f"Sentiment: {sentiment_label}")

In [ ]:
text = input("Enter text for sentiment analysis: ")
sentiment = predict_sentiment(text)
sentiment_label = "Positive" if sentiment == 1 else "Negative"
print(f"Sentiment: {sentiment_label}")

Enter text for sentiment analysis: this is a good example of bad movie
Sentiment: Negative
